## <font color='black'>191024 Microcoleus Isolates Whole Genome Sequence Analysis</font>

### 1. trimming

In [ ]:
declare -a prefix=("01" "02" "03" "04" "05" "06" "07" "08" "09" "10" "11")
declare -a sample=("S5" "S6" "S7" "S8" "S9" "S10" "S11" "S12" "S13" "S14" "S15")

for i in {0..10};
do
    # Identify the correct values
    p=${prefix[ $i ]}
    s=${sample[ $i ]}

module load BBMap/37.93-gimkl-2017a
##1a.2. Quality trim reads

mkdir 1.bbduk

bbduk.sh in1=0.raw/H7M5HBCX3-5203-${p}-00-01_${s}_L002_R1_001.fastq.gz in2=0.raw/H7M5HBCX3-5203-${p}-00-01_${s}_L002_R2_001.fastq.gz \
 out1=clean.${p}.1.fastq out2=clean${p}.2.fastq ref=/opt/nesi/mahuika/BBMap/37.93-gimkl-2017a/resources/adapters.fa \
  overwrite=true hdist=1 ktrim=r ordered minlen=80 minlenfraction=0.33 mink=11 tbo tpe rcomp=f k=23 ow=t 

sleep 5
bbduk.sh qtrim=rl trimq=30 minlen=80 in1=clean.${p}.1.fastq in2=clean${p}.2.fastq out1=t.${p}.1.fastq out2=t.${p}.2.fastq outs=t.single.trim.${p}.fastq overwrite=true;done

mv clean* t.* 1.bbduk


### 2. fastqc

In [ ]:
cd 1.bbduk;
mkdir fastqc

for i in clean*.fastq; do fastqc $i -o fastqc-files -f fastq -t 2; done
for i in t.*.fastq; do fastqc $i -o fastqc-files -f fastq -t 2; done



### 3. assembly 

In [ ]:
module load SPAdes/3.13.1-gimkl-2018b

mkdir 2-metaspades

cd 1.bbduk

export FILES=($(ls t*.1.fastq))

FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}

export NAME=$(echo ${FILENAME} | sed -e 's/\(.*\).[1/2].fastq/\1/g')

srun metaspades.py -m 100 -t 32 -k 41,61,81,101,127 --pe1-1 $NAME.1.fastq --pe1-2 $NAME.2.fastq --pe1-s $NAME.single.fastq -o ../2-metaspades/st.$NAME.k41-k127


for i in {0..11};do mv slurm.2.spades.$i.out slurm.2.spades_$i.out;done


### 4. spade stats

In [ ]:
mkdir 2-stats-metaspades


export FILE=m2000
export FILE2=scaffolds
export FILE3=contigs

for i in {01..11}; do 
#ruby ~/scripts/extract_contigs_by_length_greater.rb s.t.$i.k41-k127/${FILE3}.fasta 2000 > 2-stats-metaspades/$i.Filter.${FILE}.fa;
#ruby ~/scripts/fasta_contig_len_distribution.rb $i.Filter.${FILE}.fasta > 2-stats-metaspades/stats.$i.Filter.${FILE}.fa.txt;
##Contig >2kb extract (st):
ruby ~/scripts/extract_contigs_by_length_greater.rb 2-metaspades/s.t.$i.k41-k127/${FILE2}.fasta 2000 > 2-stats-metaspades/$i.Filter.${FILE}.fa;
##Contig stats (st):
ruby ~/scripts/fasta_contig_len_distribution.rb 2-stats-metaspades/$i.Filter.${FILE}.fa > 2-stats-metaspades/stats.$i.Filter.${FILE}.fa.txt;
done


### 5. insert size distribution

In [ ]:
grep 'Insert size' slurm.2.spades* >3-metaspades/insert_size.txt

mkdir 5-insert-size
mkdir 5-insert-size/bowtie_contigs_meta

cd 2-trimseqs; 
export FILES=($(ls t*.1.fastq))

FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}

export NAME=$(echo ${FILENAME} | sed -e 's/t.\(.*\).[1/2].fastq/\1/g')


sleep 5 

module load Bowtie/1.2.0-gimkl-2017a #module load Bowtie/1.1.2-intel-2015a
cd ../
srun bowtie-build -f 5-stats-metaspades/contigs.$NAME.k41-k127.fa 5-insert-size/bowtie_contigs_meta/bowtie_index_contigs$NAME

sleep 5


srun bowtie --phred33-quals -t -p 5 -n 1 -l 222 --minins 200 --maxins 800 --best \
--sam 5-insert-size/bowtie_contigs_meta/bowtie_index_contigs$NAME \
-1 2-trimseqs/t.$NAME.1.fastq -2 2-trimseqs/t.$NAME.2.fastq > 5-insert-size/bowtie_contigs/$NAME.sam \
2> 5-insert-size/bowtie_contigs/$NAME.log

sleep 5

grep -v '*' 5-insert-size/bowtie_contigs_meta/$NAME.sam > 5-insert-size/bowtie_contigs_meta/mapped.$NAME.sam

sleep 5

rm 5-insert-size/bowtie_contigs_meta/$NAME.sam

cd 5-insert-size/bowtie_contigs_meta

sleep 5

srun ruby ~/scripts/mapped_read_stats.rb mapped.$NAME.sam contigs.$NAME.k41-k127.fa --none

sleep 5

srun grep -E '^:|@' rb_mapping_stats_contigs.$NAME.k41-k127.fa.txt > mapped_coverage_contigs.$NAME.k41-k127.txt

srun ruby ~/scripts/fasta_contig_length-gc.rb ../../5-stats-metaspades/contigs.$NAME.k41-k127.fa > gc.contigs.$NAME.k41-k127.fa.txt
cd ../../

#### 5.2 Get ORFs and predicted protein sequences:

In [ ]:
module load prodigal/2.6.3-GCCcore-7.4.0

export FILES=($(ls -1 2-stats-metaspades/{01..11}.Filter.m2000.fa)) 

FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g')

srun prodigal -i 2-stats-metaspades/$NAME.Filter.m2000.fa -o $NAME.genes -a $NAME.genes.faa -d $NAME.genes.fna -m -p met

mkdir 5.prodigal_meta

mv $NAME* 5.prodigal_meta/

#### 5.3 GC COVERAGE Plot

In [ ]:
for i in *genes.faa;do ruby ~/scripts/fasta_contig_length-gc.rb $i > gc.$i.txt; done
for i in gc.*;do sed -i 's/_cov_\([0-9]\.[0-9]*\)_.*_.*_.*_/_cov\t\1\t/g; s/cont=//g; s/Length.*//g' $i;done

#### 5.4 Count read length

In [ ]:
cat Filt.S1_L1_R1.fastq | awk '{if(NR%4==2) print length($1)}' | sort -n | uniq -c > read_length.txt

### 6. EMIRGE (reconstruct full length ribosomal genes and abundance)

#### 6.1. EMIRGE directory:


### mkdir 3-emirge

#### 6.2. EMIRGE database and run: (more cpu less memory)

In [ ]:
module load SAMtools/0.1.19-gimkl-2017a
module load Python/2.7.14-gimkl-2017a

In [ ]:
module load SAMtools/0.1.19-gimkl-2017a
module load Python/2.7.14-gimkl-2017a

#emirge sbatch (more cores less memory)
mkdir 3-emirge/M1; srun emirge.py 3-emirge/M1/ -1 1.bbduk/t.01.1.fastq -2 1.bbduk/t.01.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 502 -s 169 -n 80 -a 32 --phred33 &>3-emirge/M1.log
mkdir 3-emirge/M2; srun emirge.py 3-emirge/M2/ -1 1.bbduk/t.02.1.fastq -2 1.bbduk/t.02.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 514 -s 185 -n 80 -a 32 --phred33 &>3-emirge/M2.log
# mkdir 3-emirge/M3; srun emirge.py 3-emirge/M3/ -1 1.bbduk/t.03.1.fastq -2 1.bbduk/t.03.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 478 -s 127 -n 80 -a 32 --phred33 &>3-emirge/M3.log
 mkdir 3-emirge/M4; srun emirge.py 3-emirge/M4/ -1 1.bbduk/t.04.1.fastq -2 1.bbduk/t.04.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 439 -s 165 -n 80 -a 32 --phred33 &>3-emirge/M4.log
 mkdir 3-emirge/M5; srun emirge.py 3-emirge/M5/ -1 1.bbduk/t.05.1.fastq -2 1.bbduk/t.05.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 434 -s 163 -n 80 -a 32 --phred33 &>3-emirge/M5.log
 mkdir 3-emirge/M6; srun emirge.py 3-emirge/M6/ -1 1.bbduk/t.06.1.fastq -2 1.bbduk/t.06.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 431 -s 166 -n 80 -a 32 --phred33 &>3-emirge/M6.log
 mkdir 3-emirge/M7; srun emirge.py 3-emirge/M7/ -1 1.bbduk/t.07.1.fastq -2 1.bbduk/t.07.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 392 -s 155 -n 80 -a 32 --phred33 &>3-emirge/M7.log
 mkdir 3-emirge/M8; srun emirge.py 3-emirge/M8/ -1 1.bbduk/t.08.1.fastq -2 1.bbduk/t.08.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 501 -s 171 -n 80 -a 32 --phred33 &>3-emirge/M8.log
 #mkdir 3-emirge/M9; srun emirge.py 3-emirge/M9/ -1 1.bbduk/t.09.1.fastq -2 1.bbduk/t.09.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 478 -s 127 -n 80 -a 32 --phred33 &>3-emirge/M9.log
 mkdir 3-emirge/M10; srun emirge.py 3-emirge/M10/ -1 1.bbduk/t.10.1.fastq -2 1.bbduk/t.10.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 432 -s 139 -n 80 -a 32 --phred33 &>3-emirge/M10.log
 mkdir 3-emirge/M11; srun emirge.py 3-emirge/M11/ -1 1.bbduk/t.11.1.fastq -2 1.bbduk/t.11.2.fastq \
 -f ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed.fasta \
 -b ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.fixed \
 -l 251 -i 421 -s 144 -n 80 -a 32 --phred33 &>3-emirge/M11.log


##### 6.2.1 Multiple sbatch

In [ ]:
for i in {01..11};do sed 's/S1/$i/g' 6a.emirgeS1.sbatch >6a.emirge$i.sbatch;done

In [ ]:
for i in 3*sbatch;do sbatch $i;done

#### 6.3. Rename: (sbatch) 6b.emirge.rename.sbatch

#### 6.4 Get number of sequences reconstructed per sample:

In [ ]:
for i in 3-emirge/*.i80.fasta; do echo "#$i"; grep '>' "$i" | wc -l; done

#3-emirge/M10.i80.fasta
5
#3-emirge/M11.i80.fasta
7
#3-emirge/M1.i80.fasta
7
#3-emirge/M2.i80.fasta
5
#3-emirge/M3.i80.fasta
5
#3-emirge/M4.i80.fasta
7
#3-emirge/M5.i80.fasta
5
#3-emirge/M6.i80.fasta
8
#3-emirge/M7.i80.fasta
5
#3-emirge/M8.i80.fasta
5
#3-emirge/M9.i80.fasta
3



#### 6.5. EMRIGE: search and annotate (#)use blast not usearch, MAKE SURE TO USE EMIRGE.RENAME.PY

##### 6.5.1. Prep databases:

In [ ]:
module load BLAST/2.6.0-gimkl-2017a
makeblastdb -in SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta -dbtype nucl \
-out ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.blastdb

##### 6.5.2 Search using blast:

In [ ]:
module load BLAST
for i in 3-emirge/*i80.fasta;do blastn -query $i \
-db ~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.ge1200bp.le2000bp.0.97.blastdb \
-num_threads 12 -outfmt 6 -num_alignments 1 -culling_limit 1 -out $i.SSU132.bln6.txt;done

cd 3-emirge

for i in *.bln6.txt;do NAME=$(echo ${i} | sed -e 's/\(.*\).i80.fasta.SSU132.bln6.txt/\1/g');\
ruby ~/scripts/annot_blast_emirge_cov.rb $i \
~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta ${NAME}.i80.fasta \
-s ${NAME}.log -l 251;done

or 

ruby ~/scripts/annot_blast_emirge_cov.rb emirgetest.SSU132.bln6.txt  \
~/database/Databases/SILVA_emirge/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta iter.80.cons.fasta \
-s emirge.bbmap.log -l 251;done


##### 6.5.3. Combine annotated output:


In [ ]:
cat annot.*.i80.fasta.SSU132.bln6.txt >annot.emirge.ball.i80-SSU132.bln6.txt;cd ../

### 7. EMRIGE OTU-table: Cluster and annotate emirge hierarchically (use own emirge-convert-priors.rb)

In [ ]:
mkdir 4-clust-emirge 

#### 7.1. Remove N's #skip this step as M7 has N

In [ ]:

cd 3-emirge; for i in *fasta; do ruby ~/scripts/extract_seqbyNs.rb "$i" 10 > ../4-clust-emirge/nn."$i"; done
cd ../4-clust-emirge

#### 7.2. Sequentially number fasta headers



In [ ]:
for i in nn.*.fasta; do ruby ~/scripts/fasta-add-seq-numbering.rb "$i" @; done

#### 7.3. Rename fasta headers with sample info

In [ ]:
for i in r.nn.*.fasta; do a=$(echo "$i" | sed 's/r.nn.\(.*\).i80.fasta/\1/g'); echo $a; sed 's/>.*Prior=\(.*\) Length=\(.*\) NormPrior=\(.*\)@\(.*\)/>'"$a"'.\4;prior=\1;length=\2;normprior=\3;size=\3;/g' "$i" > r"$i"; done
for i in rr.nn.*.fasta; do a=$(echo "$i" | sed 's/rr.nn.\(.*\).i80.fasta/\1/g'); echo $a; sed 's/>.*Prior=\(.*\) Length=\(.*\) NormPrior=\(.*\)@\(.*\)/>'"$a"'.\4;size=\3;/g' "$i" > normprior.emirge."$a".i80.fasta; done


#### 7.4. Fasta with read coverage (cov=norm only to seq length; cov.norm=also norm by number mapped reads across samples - norm_seq_mid = seq_mid.to_f * (avelog.to_f / totmapreads.to_f))

In [ ]:
for i in normprior.*.fasta; do a=$(echo "$i" | sed 's/normprior.emirge.\(.*\).i80.fasta/\1/g'); echo $a;\
ruby ~/szescript/emirge-convert-priors.rb "$i" ../3-emirge/"$a".log rcov.emirge."$a" log; done

cat summary-rcov.emirge*.log > all-summary-emirge.log

for i in normprior.*.fasta; do a=$(echo "$i" | sed 's/normprior.emirge.\(.*\).i80.fasta/\1/g'); echo $a;\
ruby ~/szescript/emirge-convert-priors.rb "$i" ../3-emirge/"$a".log "$a" 251 all-summary-emirge.log; done

#### 7.5. Concatenate fastas

In [ ]:
cat normprior.emirge.*.fasta > all.normprior.emirge.fasta
cat cov.*.fasta > all.cov.emirge.fasta
cat cov.norm.*.fasta > all.cov.norm.emirge.fasta

#### 7.6. Scale normpriors to

In [ ]:
ruby ~/scripts/emirge-scale-priors.rb all.normprior.emirge.fasta all.scale.normprior.emirge
rm all.normprior.emirge.fasta

#### 7.7. Rename coverage/normprior headers for uclust: 'prior' to 'size'

In [ ]:
sed -i 's/prior=/size=/g; s/\(>.*\)/\1;/g' all.cov*emirge.fasta

#### 7.8. Sort by length:

In [ ]:
for i in all.*.emirge.fasta; do usearch -sortbylength "$i" -fastaout sort."$i" -minseqlength 500; done

#### 7.9. Non-global clustering with UCLUST (partial matches allowed): 97 and 100%

In [ ]:
#https://www.drive5.com/usearch/manual/uclust_algo.html
for i in sort.all.*.emirge.fasta; do usearch -cluster_fast "$i" -id 0.97 -centroids clust.97."$i"; done
for i in sort.all.*.emirge.fasta; do usearch -cluster_fast "$i" -id 1.00 -centroids clust.100."$i"; done

#### 7.10. Generate OTU table:

In [ ]:
for i in clust.97.sort.all.*.emirge.fasta; do a=$(echo $i | sed 's/clust.*.sort.\(.*\).fasta/\1/g');\
b=$(echo $i | sed 's/clust.\(.*\).sort.*.fasta/\1/g'); usearch -usearch_global sort."$a".fasta -db "$i" \
-strand plus -id 0.97 -otutabout otu-table."$b"."$a".txt; done

for i in clust.100.sort.all.*.emirge.fasta; do a=$(echo $i | sed 's/clust.*.sort.\(.*\).fasta/\1/g');\
b=$(echo $i | sed 's/clust.\(.*\).sort.*.fasta/\1/g'); usearch -usearch_global sort."$a".fasta -db "$i" \
-strand plus -id 1.00 -otutabout otu-table."$b"."$a".txt; done


#### 7.11. Annotate part 1: SINTAX

In [ ]:
for i in clust.*.fasta; do usearch -sintax "$i" -db /home/htee663/database/Databases/SILVA_132_NR99/silva132_99.utax.udb \
-tabbedout "$i".sintax -strand both -sintax_cutoff 0.8 -threads 36; done


#### 7.12. Annotate part 2: Add taxa to OTU table ****and remove rows summing to <2****

In [ ]:
for i in clust.*.sintax; do NAME=$(echo $i | sed 's/clust.\(.*\).sort.\(.*\).fasta.sintax/\1.\2/g'); \
echo ${NAME}; ruby ~/szescript/annot-sintax-otutable-nopath.rb "$i" otu-table.${NAME}.txt 0; done


## 8. Rarefaction + R plotting to do

## 9. Binning

### 9.1 bowtie mapping

In [ ]:
mkdir 6a-metaspades-mapped-coverage-binning

export FILES=($(ls -1 2-stats-metaspades/scaffold.*.fa))
export FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export DIR1=$(echo ${FILENAME} | sed -e 's/\(2-stats-metaspades\)\/.*/\1/g')
export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(scaffold.*\).fa/\1/g')
export SAMPLE=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/scaffold.i.\(b.*\)\..*.fa/\1/g')
export DIR2=6a-metaspades-mapped-coverage-binning

##Mapping: --mem-per-cpu=2000 --cpus-per-task=5
for FILENAME in 2-stats-metaspades/*m2000.fa;do export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g'); mkdir ${DIR2}/bowtie_${NAME}; bowtie-build -f ${FILENAME} ${DIR2}/bowtie_${NAME}/bowtie_index_contigs;done


export DIR1=6a-metaspades-mapped-coverage-binning
export DIR2=1.bbduk
##Mapping: --mem-per-cpu=2000 --cpus-per-task=5
for FILENAME in 6a-metaspades-mapped-coverage-binning/*;do export NAME=$(echo $FILENAME | sed -e 's/6a-metaspades-mapped-coverage-binning\/bowtie_//g'); \
 bowtie --phred33-quals -t -p 5 -n 1 -l 251 --minins 200 --maxins 800 --best --sam ${FILENAME}/bowtie_index_contigs \
 -1 ${DIR2}/t.${NAME}.1.fastq -2 ${DIR2}/t.${NAME}.2.fastq> ${DIR1}/${NAME}.sam 2> ${DIR1}/${NAME}.log; done

sleep 5

for FILENAME in 6a-metaspades-mapped-coverage-binning/*.sam;do export NAME=$(echo $FILENAME | sed -e 's/6a-metaspades-mapped-coverage-binning\/\(.*\).sam//g'); \
 grep -v '*' $FILENAME > 6a-metaspades-mapped-coverage-binning/mapped.$NAME.sam;done


In [ ]:
#mapping stats
export DIR1=6a-metaspades-mapped-coverage-binning
export DIR2=1.bbduk
export a=($(ls 6a-metaspades-mapped-coverage-binning/|grep 'bowtie_'))

FILENAME=${a[$SLURM_ARRAY_TASK_ID]}
export NAME=$(echo $FILENAME | sed -e 's/bowtie_//g')

cd ${DIR1}/
srun ruby ~/scripts/mapped_read_stats.rb mapped.${NAME}.sam ${NAME} --none
srun grep -E '^:|@' rb_mapping_stats_${NAME}.txt > mapped.coverage.${NAME}.txt

In [ ]:
###9a. Mapped reads
export FILES=($(ls -1 6a-metaspades-mapped-coverage-binning/mapped*.sam))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}

export DIR1=6a-metaspades-mapped-coverage-binning
export NAME=$(echo ${FILENAME} | sed -e 's/6a-metaspades-mapped-coverage-binning\/\(mapped.*\).sam/\1/g')

module load SAMtools/1.3.1-gimkl-2017a
srun samtools view -b ${FILENAME} -o ${DIR1}/${NAME}.bam
srun samtools sort ${DIR1}/${NAME}.bam -o ${DIR1}/sort.${NAME}.bam -T ${DIR1}/temp.sort.${NAME}.bam



### <font color='black'>9.2 Metabat</font>

In [ ]:
#Metabat
mkdir 6b-metaspades-bin-metabat

###9b. MetaBAT prep
export DIR1=6b-metaspades-bin-metabat
export DIR2=6a-metaspades-mapped-coverage-binning
export FILES=($(ls -1 2-stats-metaspades/*m2000.fa))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g')

srun /home/htee663/database/Software/metabat_v2.12.1/jgi_summarize_bam_contig_depths --outputDepth ${DIR1}/depth.${NAME}.txt ${DIR2}/sort.mapped.${NAME}.bam


###c. MetaBAT bin
export DIR1=6b-metaspades-bin-metabat
export FILES=($(ls -1 2-stats-metaspades/*.fa))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g')

module load MetaBAT/2.13-GCC-7.4.0


#srun metabat2 -t 5 -s 100000 -i ${FILENAME} -a ${DIR1}/depth.${NAME}.txt -o ${DIR1}/metabat1-${NAME}/bin.${NAME} --unbinned ${DIR1}/metabat1-${NAME}/bin.${NAME} #--saveCls ${DIR1}/metabat1-${NAME}/bin.${NAME}
for i in 2-stats-metaspades/*.fa;do export NAME=$(echo ${i} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g'); \
mkdir ${DIR1}/metabat2-${NAME};  /home/htee663/database/Software/metabat_v2.12.1/metabat2 -t 20 -s 50000 -i ${i} -a 6b-metaspades-bin-metabat/depth.${NAME}.txt -o 6b-metaspades-bin-metabat/metabat2-${NAME}/bin.${NAME} --unbinned ${DIR1}/metabat2-${NAME}/bin.${NAME};done

##Summarize bins into lists: tab delimited (sample\tbin.*\tnode)
for a in metabat2-*; do cd $a; n=$(echo "$a" | sed 's/metabat2-/metabat2./g'); for i in bin.*.fa; do s=$(echo "$i" | sed 's/bin.\(.*\)\.\(.*\)\.fa/\1/g'); b=$(echo $i | sed 's/bin.\(.*\)\.\(.*\)\.fa/\2/g'); echo "$s"; grep '>' $i | sed 's/>/'"$s"'\tbin.'"$b"'\t/g' > list."$i"; done && cat list.* > ../list.$n.bins.txt; cd ../; done


### 9.3 Maxbin

In [ ]:
###9c. Binning - MaxBin: metaspades
mkdir 6c-metaspades-bin-maxbin
##Prepare coverage files:
cp 6a-metaspades-mapped-coverage-binning/mapped.coverage.* 6c-spades-bin-maxbin/
cd 6c-metaspades-bin-maxbin/
for i in mapped.*.txt; do grep '@' "$i" | cut -f2,6 > cov."$i"; done;cd ../
##Get complete contig list and assign coverage zero if missing from mapped file
for i in 2-stats-metaspades/*.fa; do a=$(echo "$i" | sed 's/2-.*\/\(.*fa\)/\1/g'); echo $a; grep '>' "$i" > 6c-metaspades-bin-maxbin/node.list."$a".txt; done
for i in 6c-metaspades-bin-maxbin/node.list.*;do sed -i 's/>//g' $i;done
cd 6c-metaspades-bin-maxbin/
for i in cov.mapped.*.txt; do a=$(echo "$i" | sed 's/cov.mapped.coverage.\([^\.]*\).txt/\1/g'); echo $a; ruby ~/scripts/match.contigs.contigs-coverage.rb node.list."$a".Filter.m2000.fa.txt "$i" all."$i"; done
##Run MaxBin array - submit from login node: ###Inconsistencies with finding software paths


export DIR1=6c-metaspades-bin-maxbin
export FILES=($(ls -1 2-stats-metaspades/*.fa))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g')

srun /home/htee663/database/Software/maxbin_v2.2.4/run_MaxBin.pl -contig ${FILENAME} -abund ${DIR1}/all.cov.mapped.coverage.*.txt -out ${DIR1}/maxbin.scaffolds.${NAME}.txt -thread 2

or 
#for i in  2-stats-metaspades/*.fa;do export NAME=$(echo ${FILENAME} | sed -e 's/2-stats-metaspades\/\(.*\).Filter.m2000.fa/\1/g');perl /home/htee663/database/Software/maxbin_v2.2.4/run_MaxBin.pl -contig ${i} -abund ${DIR1}/all.cov.mapped.coverage.${NAME}.txt -out ${DIR1}/maxbin.scaffolds.${NAME}.txt -thread 24;done

##Summarize bins into lists: tab delimited (sample\tbin.*\tnode)
cd 6c-metaspades-bin-maxbin
for a in maxbin_*; do cd $a; for i in maxbin.*.fasta; do s=$(echo "$i" | sed 's/maxbin.scaffolds.\(.*\).txt.\(.*\).fasta/\1/g'); b=$(echo "$i" | sed 's/maxbin.scaffolds.\(.*\).txt.\(.*\).fasta/\2/g'); echo "$s"; echo "$b"; grep '>' $i | sed 's/>/'"$s"'\tbin.'"$b"'\t/g' > list."$i"; done && cat list.*.fasta > ../list.$a.bins.txt; cd ../; done



### 9.4 CONCOCT

In [ ]:
###9d. Binning - CONCOCT: spades
mkdir 6d-metaspades-bin-concoct
##Prepare coverage files:
##9d.1. Cut contigs to equal lengths of 10kb min and <20kb:
source activate concoct_env #Run using <concoct> plus parameters #Deactivate using <source deactivate concoct_env>
#e.g.:   python $CONCOCT/scripts/cut_up_fasta.py -c 10000 -o 0 -m contigs.fa > frag.10k.contigs.fa
for i in 2-stats-metaspades/*.fa; do a=$(echo "$i" | sed 's/2-.*\/\(.*fa\)/\1/g'); echo "$a"; cut_up_fasta.py -c 10000 -o 0 -m "$i" -b 6d-metaspades-bin-concoct/frag.10k."$a".bed > 6d-metaspades-bin-concoct/frag.10k."$a"; done
##9d.2. Create the input table (containing average coverage per sample and contig)
##9d.2b. Build index
for i in 6a-metaspades-mapped-coverage-binning/sort.mapped*; do samtools index $i;done


##9d.2d. Make coverage table:
##Make coverage table

for i in 6d-metaspades-bin-concoct/frag.10k.*.bed;do a=$(echo "$i" | sed 's/6d-.*\/frag.10k.\(.*\).Filter.m2000.fa.bed/\1/g'); \
concoct_coverage_table.py $i 6a-metaspades-mapped-coverage-binning/sort.mapped.$a.bam >  6d-metaspades-bin-concoct/frag.10k.cov.$a.txt; done

# Bin the fragments
for i in 6d-metaspades-bin-concoct/frag.10k.cov.*.txt;do a=$(echo "$i" | sed 's/6d-.*\/frag.10k.cov.\(.*\).txt/\1/g');\
concoct --composition_file 6d-metaspades-bin-concoct/frag.10k.$a.Filter.m2000.fa --coverage_file $i -b concoct-output-$a;done

# Cluster the fragments back into their original form (python script rm points)
#for i in 6d-metaspades-bin-concoct/concoct-output-*; do cd $i; merge_cutup_clustering.py clustering_gt1000.csv > clustering_merged.csv;cd ../../;done
for i in 6d-metaspades-bin-concoct/concoct-output-*; do a=$(echo "$i" | sed 's/6d-.*\/concoct-output-\(.*\)/\1/g');\
sed -e 's/\(NODE_[0-9]*_length_[0-9]*_cov_[0-9]*\.[0-9]*\).*,\([0-9]*\)/\1,bin.\2/g;s/\(NODE_[0-9]*_length_[0-9]*_cov_[0-9]*\),\([0-9]*\)/\1,bin.\2/g;s/bin.\([1-9][0-9]\)$/bin.0\1/g;s/bin.\([1-9]\)$/bin.00\1/g' \
${i}/clustering_gt1000.csv | sort -k1,1 -u > $i/concoct-${a}-das.txt; done

# Create the bins
for i in 6d-metaspades-bin-concoct/concoct-output-*;do a=$(echo "$i" | sed 's/6d-.*\/concoct-output-\(.*\)/\1/g');\
mkdir $i/fasta_bins/; extract_fasta_bins.py 2-stats-metaspades/$a.Filter.m2000.fa ${i}/concoct-$a-das.txt \
--output_path ${i}/fasta_bins/; done

##Deactivate concoct:
source deactivate concoct_env


### 9.5 DasTool

In [ ]:
#source /nesi/project/uoa02469/Software/PipelineSetup/binning.sh
source /nesi/project/uoa02469/Software/PipelineSetup/genomic.sh
module load BLAST/2.6.0-gimkl-2017a
module load R/3.5.1-gimkl-2017a

#Installation of dependent R-packages:
R
repo='http://cran.us.r-project.org' #select a repository
install.packages('doMC', repos=repo, dependencies = T)
install.packages('data.table', repos=repo, dependencies = T)
install.packages('ggplot2', repos=repo, dependencies = T)
q() #quit R-session

unzip DAS_Tool-1.x.x.zip
cd ./DAS_Tool-1.x.x
R CMD INSTALL ./package/DASTool_1.x.x.tar.gz

#make new dir
mkdir 6e.metaspades-bin-dastool;cd 6e.metaspades-bin-dastool
# Create a MetaBAT input file
for i in ../6b-metaspades-bin-metabat/metabat2-*;do a=$(echo "$i" | sed 's/..\/6b-.*\/metabat2-\(.*\)/\1/g');\
grep ">" $i/*[0-9].fa | sed 's/.fa:>/\t/g' | awk 'OFS="\t" {print $2,$1}' > metabat.$a.txt;done
for i in metabat.*;do sed -i 's/..\/.*bin.*\./metabat./g;s/scaff.*.txt.//g' $i;done

# Create a MaxBin input file
for i in ../6c-metaspades-bin-maxbin/maxbin_*;do a=$(echo "$i" | sed 's/..\/6c-.*\/maxbin_\(.*\)/\1/g');\
grep ">" $i/maxbin*fasta | sed 's/.fasta:>/\t/g' | awk 'OFS="\t" {print $2,$1}' > maxbin.$a.txt;done
for i in maxbin.*;do sed  -i 's/..\/.*bin/maxbin/g;s/scaff.*.txt.//g' $i;done


# Create a CONCOCT input file
for i in ../6d-metaspades-bin-concoct/concoct-output-*;do a=$(echo "$i" | sed 's/..\/6d-.*\/concoct-output-\(.*\)/\1/g');\
grep ">" $i/fasta_bins/*.fa | sed 's/.fa:>/\t/g' | awk 'OFS="\t" {print $2,$1}' > concoct.$a.txt;done
for i in concoct.*;do sed -i 's/..\/.*bin/bin/g' $i;done

# Run DAS_Tool
for a in {01..11};do mkdir dastool_$a;DAS_Tool -i metabat.$a.txt,maxbin.$a.txt,concoct.$a.txt \
-l metabat,maxbin,concoct -t 36 --write_bins 1 --search_engine blast -c ../2-stats-metaspades/$a.Filter.m2000.fa \
--db_directory /opt/nesi/CS400_centos7_bdw/DAS_Tool/1.1.1-gimkl-2018b-R-3.6.1/ -o dastool_$a/$a;done


### 9.6 dRep

In [ ]:
# CheckM:

#pip install drep --user
    
mkdir 6f.metaspades-bin-drep;
for i in {01..11}; \
do mkdir 6f.metaspades-bin-drep/checkm_$i; checkm lineage_wf -t 20 --pplacer_threads 20 \
-x fa --tab_table -f bins_drep_checkm_$i 6e.metaspades-bin-dastool/dastool_${i}/${i}_DASTool_bins/ \
6f.metaspades-bin-drep/checkm_$i/ &> 6f.metaspades-bin-drep/checkm.$i.log;done
# Reformat the output
for i in {01..11}; \
do echo "genome,completeness,contamination" > dRep.genomeInfo_$i
cut -f1,12 bins_drep_checkm_$i| sed 's/\t/.fa,/g' > p1$i.txt
cut -f13 bins_drep_checkm_$i > p2$i.txt
paste -d "," p1$i.txt p2$i.txt | tail -n+2 >> dRep.genomeInfo_$i
rm p1* p2*

# Run dRep
dRep dereplicate --genomeInfo dRep.genomeInfo_${i} \
-g 6e.metaspades-bin-dastool/dastool_${i}/${i}_DASTool_bins/* -p 36 6f.metaspades-bin-drep/drep_${i}/; done


In [ ]:
#count genomes
for i in {01..11};do ls 6f.metaspades-bin-drep/drep_${i}/dereplicated_genomes/*fa |wc -l;done
for i in {01..11};do ls 6e.metaspades-bin-dastool/dastool_${i}/${i}_DASTool_bins/*fa |wc -l;done


In [ ]:
# Run CheckM
source ~/login.sh
mkdir 6g.metaspades-all-bin-drep;
mkdir 7.final_bin
module load Mash/2.1-gimkl-2018b

for i in {01..11};do for n in  6f.metaspades-bin-drep/drep_${i}/dereplicated_genomes/*; \
do export NAME=$(echo ${n} | sed -e 's/6f.metaspades-bin-drep.*genomes\/\(.*\)/\1/g'); \
cp ${n} 6g.metaspades-all-bin-drep/S${i}_${NAME};done;done

 mkdir 6g.metaspades-all-bin-drep/checkm; checkm lineage_wf -t 20 --pplacer_threads 20 \
-x fa --tab_table -f 6g.metaspades-all-bin-drep/bins_drep_checkm 6g.metaspades-all-bin-drep/  6g.metaspades-all-bin-drep/checkm/ &> 6g.metaspades-all-bin-drep/checkm.log


# Reformat the output
echo "genome,completeness,contamination" > 6g.metaspades-all-bin-drep/dRep.genomeInfo
cut -f1,12 6g.metaspades-all-bin-drep/bins_drep_checkm| sed 's/\t/.fa,/g' > p1.txt
cut -f13 6g.metaspades-all-bin-drep/bins_drep_checkm > p2.txt
paste -d "," p1.txt p2.txt | tail -n+2 >> 6g.metaspades-all-bin-drep/dRep.genomeInfo
rm p1* p2*

# Run dRep
ml MUMmer/3.23-gimkl-2017a
 module load Python/3.7.3-gimkl-2018b
dRep dereplicate --genomeInfo 6g.metaspades-all-bin-drep/dRep.genomeInfo -g 6g.metaspades-all-bin-drep/*.fa -p 24  7.new.final_bin/


## 10. Bin coverage

In [ ]:
####### 5. Pairwise mapping - Bin coverage calculation for differential genome coverage table
mkdir 7b-Refined-bins
### 1. Separate refined derep bins back to original sample
cd 7b-Refined-bins
mkdir all-refined-derep-bins-divided-per-sample
for i in {01..11}; do mkdir all-refined-derep-bins-divided-per-sample/S"$i" && cp ../7.final_bin/S"$i"*.fa all-refined-derep-bins-divided-per-sample/S"$i"/.; done


# Create list of binned scaffolds for each sample
for SAMPLE in {01..11};do 
cd /home/htee663/nobackup446/practice/practice/4.191024_comparative_Microcoleus/7b-Refined-bins/all-refined-derep-bins-divided-per-sample/S${SAMPLE}
for f in *.fa; do grep ">" ${f} > ${f}.txt; done
for f in *.txt; do sed -i "s/$/\t$f/" $f; done
cat *.txt > ${SAMPLE}.scaffolds2bin.txt && rm *.fa.txt
sed -i 's/.txt*//g' ${SAMPLE}.scaffolds2bin.txt | sed -i 's/>*//g' ${SAMPLE}.scaffolds2bin.txt ;done

### 2. Pairwise mapping with Bowtie (using indecies of assemblies already created step 5)
cd ../../../
mkdir 8-mapping-pairwise
for i in {01..11}; do mkdir 8-mapping-pairwise/S"$i"; done


#!/bin/bash
#SBATCH -A uoa00446
#SBATCH -J 8.bin.cov.map.array
#SBATCH --time 36:00:00
#SBATCH --mem 5GB
#SBATCH --array=01-11
#SBATCH --ntasks=1
#SBATCH --cpus-per-task 30
#SBATCH -e slurm-8.bin.cov.map.array.%A-%a.err
#SBATCH -o slurm-8.bin.cov.map.array.%A-%a.out

module load Bowtie/1.2.0-gimkl-2017a

declare -a array=(echo {01..11})

for i in {01..11};
do

srun bowtie-build -f 2-stats-metaspades/${i}.Filter.m2000.fa 8-mapping-pairwise/bowtie_index/${i}_bowtie_index

    srun bowtie --phred33-quals -t -p 30 -n 1 -l 222 --minins 200 --maxins 800 --best --sam 8-mapping-pairwise/bowtie_index/${array[$SLURM_ARRAY_TASK_ID]}_bowtie_index \
        -1 1.bbduk/t.${i}.1.fastq \
        -2 1.bbduk/t.${i}.2.fastq \
        >8-mapping-pairwise/S${array[$SLURM_ARRAY_TASK_ID]}/extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.sam \
        2> 8-mapping-pairwise/S${array[$SLURM_ARRAY_TASK_ID]}/extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.log;
done

## Run as 
sbatch 8.bin.cov.map.pairwise.sbatch

# Mapping stats

# In 8.bin.cov.map.stats.sbatch

#!/bin/bash
#SBATCH -A uoa00446
#SBATCH -J 8.bin.cov.map.stats.array
#SBATCH --time 6:00:00
#SBATCH --mem 10GB
#SBATCH --array=1-11
#SBATCH --ntasks=1
#SBATCH --cpus-per-task 2
#SBATCH -e slurm-8.bin.cov.map.stats.array.%A-%a.err
#SBATCH -o slurm-8.bin.cov.map.stats.array.%A-%a.out

declare -a array=(echo {01..11})

cd /home/htee663/nobackup446/practice/practice/4.191024_comparative_Microcoleus/8-mapping-pairwise/S${array[$SLURM_ARRAY_TASK_ID]}

sleep 5

for i in {01..11};
do 
    grep -v '*' extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.sam > map.extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.sam; 
done

##Get mapped read stats:
for i in {01..11}; 
do 
    ruby /home/htee663/scripts/mapped_read_stats.rb map.extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.sam extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta --none; 
done

##Grab coverage information (leaving stats):
for i in {01..11}; 
do
    srun grep -E '^:|@' rb_mapping_stats_extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.txt > mapped_coverage_extract.scaffolds.S${array[$SLURM_ARRAY_TASK_ID]}.vs.S${i}.fasta.txt;
done




### 3. Number of reads per contig per sample
#find read length

cd 8-mapping-pairwise
for n in {01..11};do echo $n >>all.read.length.txt;for i in S${n}/rb_mapping_stats_extract.scaffolds.S${n}.vs.S*;\ 
do awk 'NR==5' $i >> all.read.length.txt;done;done


for i in {01..11}; do cp S"$i"/mapped* . && cat *.vs.S"$i".fasta.txt > all.vs.S"$i".txt; done
for i in {01..11}; do grep '' S"$i"/
rm mapped*
rm -r S* # sam files too big

for i in {01..11}; do awk '{print $2 "\t" $4}' all.vs.S"$i".txt > all.vs.S"$i"-2col.txt; done

# Get list of scaffolds in each assembly
for i in ../2-stats-metaspades/*m2000.fa;do grep '>' $i | sed 's/>//g' > list.scaffolds.in.all.assemblies.txt; done
for i in {01..11}; do grep ">" ../2-stats-metaspades/"$i".Filter.m2000.fa > S"$i"-list.scaffolds.in.assembly.txt; done

sed -i 's/>*//g' *-list.scaffolds.in.assembly.txt
cat *-list.scaffolds.in.assembly.txt > list.scaffolds.in.all.assemblies.txt

for SAMPLE in {01..11};do 
ruby /home/htee663/scripts/match.contigs.contigs-coverage.rb list.scaffolds.in.all.assemblies.txt all.vs.S${SAMPLE}-2col.txt \
all.vs.S${SAMPLE}-node_reads.txt;done



### Extract contig length from list of scaffolds
cp list.scaffolds.in.all.assemblies.txt list.scaffolds.in.all.assemblies-contig_length.txt
sed -i 's/.*_length_\(.*\)_cov.*/\1/g' list.scaffolds.in.all.assemblies-contig_length.txt




make coverage table in R  
refer: 10.genome-coverage-table.R

## 11. GTDB (Taxonomic classification)

In [ ]:
source /nesi/project/uoa02469/Software/PipelineSetup/gtdbtk.sh
gtdbtk classify_wf  --min_perc_aa 50 --cpus 16 --genome_dir 7.final_bin/ --out_dir 9.gtdboutbins/ --extension fa

##Combine coverage table and taxonomic classification and completeness

## 12.  Genome Annotation

### 12.1 Prodigal

In [ ]:
source ~/login.sh
source /nesi/project/uoa02469/Software/PipelineSetup/gtdbtk.sh

#mkdir 11-bin-annotation
export FILES=($(ls -1 7.final_bin/S*.fa)) 

FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export NAME=$(echo ${FILES} | sed -e 's/7.final_bin\/\(S.*\).fa/\1/g')

srun prodigal -q -p single -i $FILENAME -d 11-bin-annotation/$NAME.prod.fna -a 11-bin-annotation/$NAME.prod.faa -o 11-bin-annotation/$NAME.genes.gff -f gff


sed 's/ .*//g' $NAME.prod.faa > $NAME.prod.no_metadata.faa
sed 's/ .*//g' $NAME.prod.fna > $NAME.prod.no_metadata.fna

### 12.2 usearch (uniprot, uniref100,kegg100) + hmmsearch (pfam, tigrfam, dbcan)

In [ ]:
source /nesi/project/uoa02469/Software/PipelineSetup/annotation.sh

cd 11-bin-annotation
#export FILES=($(ls -1 *metadata.faa)) 

#FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
for FILENAME in *metadata.faa;do export NAME=$(echo ${FILENAME} | sed -e 's/\(.*\).prod.no_metadata.faa/\1/g');

usearch -id 0.5 -evalue 0.001 -maxhits 10 -top_hits_only -threads 32\
                           -userfields query+target+tlo+thi+id+bits+evalue \
                           -db $UNIPROT_udb -usearch_global ${NAME}.prod.no_metadata.faa -threads 32 -userout ${NAME}.prod.uniprot.txt 

usearch -id 0.5 -evalue 0.001 -maxhits 10 -top_hits_only \
                           -userfields query+target+tlo+thi+id+bits+evalue \
                           -db $UNIREF100_udb -usearch_global ${NAME}.prod.no_metadata.faa -threads 32 -userout ${NAME}.prod.uniref100.txt

usearch -id 0.5 -evalue 0.001 -maxhits 10 -top_hits_only \
                           -userfields query+target+tlo+thi+id+bits+evalue \
                           -db $KEGG_udb -usearch_global ${NAME}.prod.no_metadata.faa -threads 32 -userout ${NAME}.prod.kegg100.txt
 

 srun hmmsearch --tblout ${NAME}.prod.pfam -E 1e-3 --cpu 36 ${PFAM_hmm} \
                  ${NAME}.prod.no_metadata.faa > /dev/null

    srun hmmsearch --tblout ${NAME}.prod.tigrfam -E 1e-3 --cpu 36 ${TIGRFAM_hmm} \
                   ${NAME}.prod.no_metadata.faa > /dev/null

    srun hmmsearch --domtblout ${NAME}.prod.dbcan -E 0.1 --cpu 36 ~/nobackup446/practice/dbcan/db/dbCAN.txt \
                   ${NAME}.prod.no_metadata.faa > /dev/null

    hmmscan-parser.py ${NAME}.prod.dbcan \
           > ${NAME}.prod.dbcan_summary
    


### 12.3 genomic features (Metaxa-rRNA, aragorn-non-rRNA, crisprdisco-CRISPR)

In [ ]:
#metaxa
for FILENAME in *metadata.faa;do export NAME=$(echo ${FILENAME} | sed -e 's/\(.*\).prod.no_metadata.faa/\1/g');\
    metaxa2 --cpu 2 -g ssu -i ${NAME}.prod.no_metadata.fna -o $NAME.metaxa.ssu; metaxa2 --cpu 3 -g lsu \
    -i ${NAME}.prod.no_metadata.fna -o $NAME.metaxa.lsu;

#aragorn
    aragorn -m -t -gcstd -l -a -q -rn -fon -o ${NAME}.aragorn ${NAME}.prod.no_metadata.fna; done

#crisprdisco
source activate /nesi/project/uoa02469/Software/PipelineSetup/crisprdisco_env

echo ",Path" > disco_input.csv
ls *fna >test.txt
#edit and remove extension in test.txt and combine with disco_input.csv (format : 0, genome)

disco disco_input.csv  --outdir  disco_out

### 12.4 combine all annotation (Annotation aggregator)

In [ ]:
for FILENAME in *prod.faa;do export NAME=$(echo ${FILENAME} | sed -e 's/\(.*\).prod.faa/\1/g');\
annotationAggregator.py --ident 30.0 --coverage 70.0 \
                        -u ${NAME}.prod.uniprot.txt -r ${NAME}.prod.uniref100.txt -k ${NAME}.prod.kegg.txt \
                        -p ${NAME}.prod.pfam -i ${NAME}.prod.tigrfam \
                         -t ${NAME}.aragorn \
                        ${NAME}.fa ${NAME}.prod.fna ${NAME}.prod.faa complete_annotation/${NAME}.annotation;
done